In [9]:
using Random
using Printf
using LinearAlgebra
using Statistics
using TensorCrossInterpolation
import TensorCrossInterpolation: TensorCI, CachedFunction
import TensorCrossInterpolation as TCI


In [10]:
function asian_barrier_option(j, T, K, B, S0s, sigmas, mu, r, d_normals, N_STEPS, N_PATHS)
    tmp1 = mu * T / N_STEPS
    tmp2 = exp(-r * T)
    tmp3 = sqrt(T / N_STEPS)
    d_s = zeros(Float64, N_PATHS)
    S0 = S0s[j[1]]
    sigma = sigmas[j[2]]

    
    for i in 1:N_PATHS
        s_curr = S0
        running_average = 0.0
        
        for n in 1:N_STEPS
            # Update the current price
            s_curr += tmp1 * s_curr + sigma * s_curr * tmp3 * d_normals[i + (n - 1) * N_PATHS]
            
            # Update the running average
            running_average += (s_curr - running_average) / n
            
            # Check the barrier
            if running_average <= B
                break
            end
        end
        
        # Calculate payoff
        payoff = max(running_average - K, 0)
        
        # Discounted payoff
        d_s[i] = tmp2 * payoff
    end

    return mean(d_s)
end


asian_barrier_option (generic function with 1 method)

In [11]:
# Example usage
d = 2
N_STEPS = 365
N_PATHS = 1000000
T = 1.0
K = 110.0
B = 100.0
S0 = 120.0
sigma = 0.35
mu = 0.1
r = 0.05
Random.seed!(1234)
d_normals = randn(Float64, N_STEPS * N_PATHS)  # Standard normal random numbers
S0_values = range(90, stop=120, length=101)[1:end-1]
implied_vol_values = range(0.15, stop=0.25, length=101)[1:end-1]
N = 100
localdims_v = fill(N, d)
firstpivot_v = rand(1:N, d)
#mean_ds = asian_barrier_option(j, T, K, B, S0_values, implied_vol_values, mu, r, d_normals, N_STEPS, N_PATHS)


2-element Vector{Int64}:
 89
 99

In [13]:
abo(j) = asian_barrier_option(j, T, K, B, S0_values, implied_vol_values, mu, r, d_normals, N_STEPS, N_PATHS)
# Display a sample of results
abo_ci = TCI.CachedFunction{ComplexF64}(abo, localdims_v)
#@printf("Sample of computed option prices: %0.4f\n", mean_ds)
tol = 1e-5

1.0e-5

In [14]:
function tci_oneshot(func, d, localdims, firstpivot, tol)
    BLAS.set_num_threads(4)
    #func_q = GenNDfunc(d, func)

    for isearch in 1:100
        p = TCI.optfirstpivot(func, localdims, firstpivot) # search optimal fist pivot
        if abs(func(p)) > abs(func(firstpivot))
            firstpivot = p
        end
    end

    # execute tci2
    qtt, ranks, errors = TCI.crossinterpolate2(
        ComplexF64,
        func, 
        localdims, 
        [firstpivot], 
        tolerance = tol, 
        maxiter = 5, 
        verbosity = 1, 
        loginterval = 1,
        pivotsearch = :rook,
    )

    return qtt, errors
end     

tci_oneshot (generic function with 1 method)

In [ ]:
tt_abo, errors_phi_2 = tci_oneshot(abo_ci, d, localdims_v, firstpivot_v, tol)    

In [ ]:
for i in 1:d
    @show size(tt_abo[i])
end

In [ ]:
using JLD2
JLD2.@save "tt_abo, tol$tol.jld2" tt_abo